In [1]:
import numpy as np
import bilby
from bilby.core.prior import Uniform
from bilby.core.sampler import run_sampler
from bilby.hyper.likelihood import HyperparameterLikelihood
from scipy import interpolate
import pandas as pd
from scipy import integrate
outdir='outdir'

#defalut gaussian PDF replace BILBY for posterior, N(theta,sigma_theta)
#parameter: mu=N(0,err_fisher)+mu_ture, sigma=err_fisher, prior
#return: array( mu_posterior,evidence)
class SimpleGaussianLikelihood1s(bilby.Likelihood):
    def __init__(self, data1):
        self.data1 = data1
        self.parameters = {'mu1': None, 'sigma1': None}

    def log_likelihood(self):
        mu1 = self.parameters['mu1']
        sigma1 = self.parameters['sigma1']
        res1 = self.data1 - mu1
        return -0.5 * (np.sum((res1**2) / (sigma1**2)) +
                       1* np.log(2 * np.pi * (sigma1**2)))
class SimpleGaussianLikelihood2s(bilby.Likelihood):
    def __init__(self, data2):
        self.data2 = data2
        self.parameters = {'mu2': None, 'sigma2': None}

    def log_likelihood(self):
        mu2 = self.parameters['mu2']
        sigma2 = self.parameters['sigma2']
        res2 = self.data2 - mu2
        return -0.5 * (np.sum((res2**2) / (sigma2**2)) +
                       1* np.log(2 * np.pi * (sigma2**2))) 
class SimpleGaussianLikelihood3s(bilby.Likelihood):
    def __init__(self, data3):
        self.data3 = data3
        self.parameters = {'mu3': None, 'sigma3': None}

    def log_likelihood(self):
        mu3 = self.parameters['mu3']
        sigma3 = self.parameters['sigma3']
        res3 = self.data3 - mu3
        return -0.5 * (np.sum((res3**2) / (sigma3**2)) +
                       1* np.log(2 * np.pi * (sigma3**2)))
#######################################################################
zmax=9
deltaz=0.1
c_l=3*10**5
def EZ(x,omm):
    return 1/np.sqrt((omm*(1+x)**3+(1-omm)))
def trapzez(H0,omm):
    x=np.arange(0,zmax,deltaz)
    s=np.zeros(len(x)+1)
    for i in range(len(x)):
        s[i]=np.trapz(EZ(x,omm)[0:i+1],x[0:i+1]) 
    return (s[:-1])*(1+x)*(c_l/H0)
#interpolate redshift and DL contain HO omm
def getz(DL,H0,omm):
    x=np.arange(0,zmax,deltaz)
    f=interpolate.interp1d(trapzez(H0,omm),x)
    gez=f(DL)
    return gez
def PDL(DL,H0,omm):
    z=getz(DL,H0,omm)
    return (DL*DL)/((1+z)**3)
######################################################
#mmin is the min value for m1
alpha=1.5
mmax=40
lamda=0.1
mpp=35
deltapp=1.0
beta=-0.2
mmin=8
deltam=1.901
def fm(m):
    return deltam/(m)-(deltam/((m-deltam)))
def Sm(m):
    return 1/(np.exp(fm(m-4.901))+1)
def ppow(m):
    return (m**(-1*alpha))*Sm(m)
def ppp(m):
    return np.exp(-1*(m-mpp)**2/(2*deltapp*deltapp))*Sm(m)
def Pm1(m):
    return ((1-lamda)*ppow(m)+lamda*ppp(m))
def Pm2(m2,m):
    return ((m2/m)**beta)*Sm(m2)
def normPm1(m):
    ss=np.trapz(Pm1(np.arange(mmin,40.5,0.5)),np.arange(mmin,40.5,0.5))
    return (Pm1(m)/ss)
def normPm2(m2,m):
    ss=np.trapz(Pm2(np.arange(mmin,40.5,0.5),40),np.arange(mmin,40.5,0.5))
    return Pm2(m2,m)/ss
##################################################################
choiceN =200      ############### set injection events
orlive  =800      ############### set the nlive nestle sample for posterior
kk      =1000     ############### posterior resample number 
##################################################################
#load generate mock data, set sample size form the mock data
pars=np.loadtxt('LCDMmockdata_fim_snr.txt')
dfpars=pd.DataFrame(pars, columns = ['mass_1', 'mass_2', 'a_1', 'a_2', 'tilt_1', 'tilt_2',
                     'phi_12', 'phi_jl', 'luminosity_distance', 'theta_jn', 'psi',
                     'phase', 'geocent_time', 'ra', 'dec','errm1','errm2','errDL','errl',
                     'match_filter_SNR','optimal_SNR','ratioer/t'])
#choose the snr larger 8 events
snrp=dfpars['optimal_SNR']>8
dfparsnrp=(dfpars[snrp]).reset_index(drop=True)
resamppar=(dfparsnrp.sample(choiceN)).reset_index(drop=True)
resampdata=resamppar
#####################################################################
deltadl=30 #############################step for DL 
errm1=resampdata['errm1']#####use err m1 from fisher matrix 
errm2=resampdata['errm2']#####use err m2 from fisher matrix 
errDL=resampdata['errDL']#####use err DL from fisher matrix 
max_samples=kk
maxsamp=max_samples
sampdl = resampdata['luminosity_distance']
sampz=getz(sampdl,70,0.3)
sampm1z=resampdata['mass_1']
sampm2z=resampdata['mass_2']
sampq=sampm2z/sampm1z
choice_Nevents=choiceN
Nevents=(choice_Nevents)
lambdadl=np.zeros((choiceN,max_samples))
sigmadl=np.zeros((choiceN,max_samples))
lambdam1=np.zeros((choiceN,max_samples))
sigmam1=np.zeros((choiceN,max_samples))
lambdam2=np.zeros((choiceN,max_samples))
sigmam2=np.zeros((choiceN,max_samples))
#######################################################################################
#theta=N(0,sigma_theta)+theta_True
newsampDL=np.zeros(choiceN)
newsampm1=np.zeros(choiceN)
newsampm2=np.zeros(choiceN)
newsampDL=np.array(sampdl)+np.random.normal(0,np.array(errDL))
newsampm1=np.array(sampm1z)+np.random.normal(0,np.array(errm1))
newsampm2=np.array(sampm2z)+np.random.normal(0,np.array(errm2))
normmaxDL=max(newsampDL)+max(errDL)
#######################################################################################
#Gaussian PDF for DL, N([theta_Ture+N(0,sigma_fim^2)],sigma_fim)
results1=list()
for i in range(len(sampdl)):
    data1=newsampDL[i]
    errg=errDL[i]
    lambdadl[i,:]=np.repeat(data1,maxsamp)
    sigmadl[i,:]=np.repeat(data1*errg,maxsamp)
    likelihood = SimpleGaussianLikelihood1s(data1)
    priors = dict(mu1=bilby.core.prior.Uniform(1, 31300, 'mu1'),sigma1=errg)            
    result1 = bilby.run_sampler(
                   likelihood=likelihood, priors=priors, sampler='nestle', nlive=orlive,
                   outdir=outdir, verbose=False,label='DLposterior_{}'.format(i),save=False)
    results1.append(result1)
samples1= [result1.posterior for result1 in results1]
evidences1 = [result1.log_evidence for result1 in results1]
########################################################
results2=list()
for i in range(len(sampm1z)):
    data2=newsampm1[i]
    errg=errm1[i]
    lambdam1[i,:]=np.repeat(data2,maxsamp)
    sigmam1[i,:]=np.repeat(data2*errg,maxsamp)
    likelihood = SimpleGaussianLikelihood2s(data2)
    priors = dict(mu2=bilby.core.prior.Uniform(7, 160, 'mu2'),sigma2=errg)            
    result2 = bilby.run_sampler(
                   likelihood=likelihood, priors=priors, sampler='nestle', nlive=orlive,
                   outdir=outdir, verbose=False,label='m1posterior_{}'.format(i),save=False)
    results2.append(result2)
samples2= [result2.posterior for result2 in results2]
evidences2 = [result2.log_evidence for result2 in results2]
#########################################################
results3=list()
for i in range(len(sampm2z)):
    data3=newsampm2[i]
    errg=errm2[i]
    lambdam2[i,:]=np.repeat(data3,maxsamp)
    sigmam2[i,:]=np.repeat(data3*errg,maxsamp)
    likelihood = SimpleGaussianLikelihood3s(data3)
    priors = dict(mu3=bilby.core.prior.Uniform(6.8, 124.8, 'mu3'),sigma3=errg)            
    result3 = bilby.run_sampler(
                   likelihood=likelihood, priors=priors, sampler='nestle', nlive=orlive,
                   outdir=outdir, verbose=False,label='m2posterior_{}'.format(i),save=False)
    results3.append(result3)
samples3= [result3.posterior for result3 in results3]
evidences3 = [result3.log_evidence for result3 in results3]
dfsampre=list()
for i in range(len(sampdl)):
    dfsamp1=((samples1[i]).sample(n=maxsamp)).reset_index(drop=True)
    dfsamp2=((samples2[i]).sample(n=maxsamp)).reset_index(drop=True)
    dfsamp3=((samples3[i]).sample(n=maxsamp)).reset_index(drop=True)
    dfsamp12=pd.concat([dfsamp1,dfsamp2,dfsamp3],axis=1)
    dfsamp12.columns=['mu1','sigma1','log_likelihood1','log_prior1','mu2','sigma2','log_likelihood2','log_prior2','mu3','sigma3','log_likelihood3','log_prior3']
    dfsampre.append(dfsamp12)
#combine the posterior DL m1 m2, calculated the total evidence
evidences=np.array(evidences1)+np.array(evidences2)+np.array(evidences3)
#####################################################################
#hyper prior \pi(DL,m1,m2|H0,omm,M_LCDM) in source frame
def normPDL(DL,H0,omm):
    ss=np.trapz(PDL(np.arange(0,(normmaxDL+200),100),H0,omm),np.arange(0,(normmaxDL+200),100))
    return PDL(DL,H0,omm)/ss 
def hyper_prior(data,H0,omm):
    return normPDL(data['mu1'],H0,omm)*normPm1(data['mu2']/(np.array((getz(data['mu1'],H0,omm)))+1))\
           *normPm2((data['mu3']/(np.array((getz(data['mu1'],H0,omm))))),(data['mu2']/(np.array((getz(data['mu1'],H0,omm))))))
#defalt prior \pi(DL,m1,m2|phi)
def run_prior(data):
    return 1
hp_likelihood = HyperparameterLikelihood(
    posteriors=dfsampre, hyper_prior=hyper_prior,
    sampling_prior=run_prior, log_evidences=evidences, max_samples=kk)

hp_priors = dict(H0=Uniform(30,110, 'H0', '$H0$')
                 ,omm=Uniform(0.01,0.7, 'omm', '$\Omega_m$'))
result = run_sampler(
    likelihood=hp_likelihood, priors=hp_priors, sampler='dynesty', nlive=2000,
    use_ratio=False, outdir=outdir, label='LCDM20190529N200hyper_parameter',
    verbose=True, clean=True)
result.plot_corner(truth=dict(H0=70,omm=0.3))

12:57 bilby INFO    : Running bilby version: 0.3.6: (CLEAN) 97937d1 2019-02-11 21:15:16 -0600
12:58 bilby WARNING : You do not have lalsuite installed currently. You will not be able to use some of the prebuilt functions.
12:58 bilby WARNING : You do not have lalsuite installed currently. You will not be able to use some of the prebuilt functions.
12:58 bilby WARNING : You do not have lalsuite installed currently. You will not be able to use some of the prebuilt functions.


OSError: LCDMmockdata_fim_snr.txt not found.